In [1]:
import os
import subprocess

import pandas as pd
import numpy as np

path_to_NPDtools = '/home/letovesnoi/Downloads/NPDtools-2.4.0-Linux/'

data_dir = 'data'
gnps_library = os.path.join(data_dir, 'GNPS-LIBRARY.mgf')
# gnps_library = os.path.join(data_dir, 'tst.mgf')

# create directory for MGF files
spectra_dir = os.path.join(data_dir, 'spectra')
if os.path.exists(spectra_dir):
    subprocess.call('rm -r {}'.format(spectra_dir), shell=True)
os.mkdir(spectra_dir)

# create directory for Molfiles
mols_dir = os.path.join(data_dir, 'mols')
if os.path.exists(mols_dir):
    subprocess.call('rm -r {}'.format(mols_dir), shell=True)
os.mkdir(mols_dir)

In [2]:
def get_header(fin):
    header = []
    line = fin.readline()
    while '=' in line:
        ind = line.find('=')
        header.append((line[:ind], line[ind + 1:].strip()))
        line = fin.readline()
    return header, line

def get_spectrum(fin):
    header, line = get_header(fin)
    intensity = []
    while line != 'END IONS\n':
        intensity.append(line.strip().split())
        line = fin.readline()
    return header, np.array(intensity, dtype='float64')

def get_spectrum_id(header):
    return header[18][1]

def write_spectrum(header, intensity, spectrum_path):
    with open(spectrum_path, 'a') as fout:
        fout.write('BEGIN IONS\n')
        for key, value in header:
            fout.write(key + '=' + value + '\n')
        for mass, abundance in intensity:
            fout.write(str(mass) + ' ' + str(abundance) + '\n')
        fout.write('END IONS\n')

def split_mgf(gnps_library):
    spectra_pathes = []
    with open(gnps_library, 'r') as fin:
        num_spectra = 0
        while True:
            line = fin.readline()
            if not line:
                break
            num_spectra += 1
            header, intensity = get_spectrum(fin)
            spectra_pathes.append(os.path.join(spectra_dir, get_spectrum_id(header) + '.mgf'))
            write_spectrum(header, intensity, spectra_pathes[-1])
    # print('{} spectra'.format(num_spectra))
    return spectra_pathes

In [3]:
peptidic = 0
smiles_errors = 0
known = 0
empty = 0

def get_smiles(header):
    return header[11][1]

def get_mol(spectrum_path):
    with open(spectrum_path, 'r') as fin:
        fin.readline()
        header, intensity = get_spectrum(fin)
        molfile = os.path.join(mols_dir, get_spectrum_id(header) + '.mol')
        # print('SMILES=' + get_smiles(header))
        # print('SPECTRUMID=' + get_spectrum_id(header))
        subprocess.call('molconvert mol:V3+H -s \'{}\' -o {}'.format(get_smiles(header), molfile), shell=True)
    return molfile

def is_structural(mgf_file):
    global known
    with open(mgf_file, 'r') as fin:
        fin.readline()
        header, intensity = get_spectrum(fin)
        if get_smiles(header) == 'N/A' or get_smiles(header) == '':
            return False
        else:
            known += 1
            return True

def is_peptidic(molfile):
    global smiles_errors, peptidic
    tmp_peptidic = os.path.join(data_dir, 'peptidic.tmp')
    subprocess.call('{0}bin/print_structure {1} -C {0}share/npdtools/ --print_rule_fragmented_graph > {2}'.format(path_to_NPDtools, molfile, tmp_peptidic), shell=True)
    with open(tmp_peptidic, 'r') as fin:
        line = fin.readline()
        subprocess.call('rm {}'.format(tmp_peptidic), shell=True)
        if 'number of components' in line:
            num_components = line.strip().split(':')[1][1:]
        else:
            smiles_errors += 1
            print('{} contains an error'.format(molfile))
            return -1
    if int(num_components) > 3:
        peptidic += 1
        return 1
    else:
        return 0

def is_empty(mgf_file):
    global empty
    with open(mgf_file, 'r') as fin:
        fin.readline()
        header, intensity = get_spectrum(fin)
        if len(intensity) != 0:
            return False
        else:
            empty += 1
            print('{} has no information about intensity'.format(mgf_file))
            return True
        
def filter_spectra():
    for mgf_file in split_mgf(gnps_library):
        # remove unknown spectrum
        if not is_structural(mgf_file) or is_empty(mgf_file):
            subprocess.call('rm {}'.format(mgf_file), shell=True)
        else:
            molfile = get_mol(mgf_file)
            # remove non-peptidic data
            is_pep = is_peptidic(molfile)
            if is_pep == 0 or is_pep == -1:
                subprocess.call('rm {}'.format(mgf_file), shell=True)
                subprocess.call('rm {}'.format(molfile), shell=True)
                
filter_spectra()

print('{} have a known structure'.format(known))
print('{} have no information about intensity'.format(empty))
print('{} SMILES contains an error'.format(smiles_errors))
print('{} peptidic among structural, with non zero intensity, and with correct SMILES'.format(peptidic))

data/spectra/CCMSLIB00000078897.mgf has no information about intensity
data/mols/CCMSLIB00000478055.mol contains an error
data/mols/CCMSLIB00000478057.mol contains an error
data/mols/CCMSLIB00000478082.mol contains an error
data/mols/CCMSLIB00000478089.mol contains an error
data/mols/CCMSLIB00000478097.mol contains an error
data/mols/CCMSLIB00000478103.mol contains an error
data/spectra/CCMSLIB00000577483.mgf has no information about intensity
data/spectra/CCMSLIB00000577484.mgf has no information about intensity
data/spectra/CCMSLIB00000577485.mgf has no information about intensity
data/spectra/CCMSLIB00004722219.mgf has no information about intensity
data/mols/CCMSLIB00005435738.mol contains an error
data/mols/CCMSLIB00005436087.mol contains an error
2247 have a known structure
5 have no information about intensity
8 SMILES contains an error
443 peptidic among structural, with non zero intensity, and with correct SMILES


In [4]:
# linear, cyclic, branch-cyclic or complex
def get_spectrum_cyclicality(molfile):
    tmp_cyclicality = os.path.join(data_dir, 'cyclicality.tmp')
    subprocess.call('{0}bin/print_structure {1} -C {0}share/npdtools/ --print_structure > {2}'.format(path_to_NPDtools, molfile, tmp_cyclicality), shell=True)
    with open(tmp_cyclicality, 'r') as fin:
        cyclicality = fin.readline().strip()
    subprocess.call('rm {}'.format(tmp_cyclicality), shell=True)
    return cyclicality

def get_csv(spectra_pathes):
    csv_path = os.path.join(data_dir, 'data.csv')
    discrete_masses = np.linspace(0, 5000, num=50000, dtype='float64')
    spectra_df = pd.DataFrame()
    for mgf_file in spectra_pathes:
        with open(mgf_file, 'r') as fin:
            fin.readline()
            header, intensity = get_spectrum(fin)
        id = os.path.splitext(os.path.basename(mgf_file))[0]
        bins = pd.cut(intensity[:, 0], bins=discrete_masses)
        spectrum_df = pd.DataFrame({id: intensity[:, 1], 'binned': bins}).groupby(['binned']).sum().T
        spectra_df = spectra_df.append(spectrum_df)
    spectra_df.to_csv(csv_path)
    return csv_path

def get_labels():
    labels = []
    for molfile in os.listdir(mols_dir):
        labels.append(get_spectrum_cyclicality(os.path.join(mols_dir, molfile)))
    return labels

spectra_csv = get_csv([os.path.join(spectra_dir, spectrum_path) for spectrum_path in os.listdir(spectra_dir)])
spectra_df = pd.read_csv(spectra_csv)
display(spectra_df)

labels = get_labels()
print('{} linear'.format(labels.count('linear')))
print('{} cyclic'.format(labels.count('cyclic')))
print('{} branch-cyclic'.format(labels.count('branch-cyclic')))
print('{} complex'.format(labels.count('complex')))

,Unnamed: 0,"(0.0, 0.1]","(0.1, 0.2]","(0.2, 0.3]","(0.3, 0.4]","(0.4, 0.5]","(0.5, 0.6]","(0.6, 0.7]","(0.7, 0.8]","(0.8, 0.9]",...,"(4999.0, 4999.1]","(4999.1, 4999.2]","(4999.2, 4999.3]","(4999.3, 4999.4]","(4999.4, 4999.5]","(4999.5, 4999.6]","(4999.6, 4999.7]","(4999.7, 4999.8]","(4999.8, 4999.9]","(4999.9, 5000.0]"
0,CCMSLIB00000081207,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,CCMSLIB00000839208,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,CCMSLIB00000840332,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,CCMSLIB00000424850,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,CCMSLIB00005435909,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
438,CCMSLIB00000839204,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
439,CCMSLIB00000478601,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
440,CCMSLIB00000075323,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
441,CCMSLIB00000070304,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


85 linear
82 cyclic
71 branch-cyclic
205 complex


In [ ]:
reg_run_dir = os.path.join(data_dir, 'REG_RUN_GNPS')
fdr0_mgf = os.path.join(reg_run_dir, 'REG_fdr0_spectra')
fdr0_tsv = os.path.join(reg_run_dir, 'regrun_fdr0_complete.tsv')

def split_mgf_by_ids(mgf_library, spectra_dir, local_ids):
    spectra_pathes = []
    with open(mgf_library, 'r') as fin:
        num_spectra = 0
        while True:
            line = fin.readline()
            if not line:
                break
            header, intensity = get_spectrum(fin)
            if num_spectra in local_ids:
                mgf_name = os.path.splitext(os.path.basename(mgf_library))[0]
                spectra_pathes.append(os.path.join(spectra_dir, mgf_name + '_' + str(num_spectra) + '.mgf'))
                write_spectrum(header, intensity, spectra_pathes[-1])
            num_spectra += 1
    return spectra_pathes

# create directory for MGF files
spectra_rr_dir = os.path.join(data_dir, 'spectra_REG_RUN')
if os.path.exists(spectra_rr_dir):
    subprocess.call('rm -r {}'.format(spectra_rr_dir), shell=True)
os.mkdir(spectra_rr_dir)

reg_run_df = pd.read_csv(fdr0_tsv, sep="\t")

grouped_df = reg_run_df.groupby(['SpecFile'])['LocalSpecIdx', 'Structure'].agg(list)

spectra_rr_pathes = []
labels_rr = []
for index, row in grouped_df.iterrows():
    mgf_path = os.path.join(reg_run_dir, os.path.splitext(index)[0] + '_filtered.mgf')
    spectra_rr_pathes.extend(split_mgf_by_ids(mgf_path, spectra_rr_dir, row['LocalSpecIdx']))
    labels_rr.extend(row['LocalSpecIdx'])
    
spectra_rr_csv = get_csv(spectra_rr_pathes)
spectra_rr_df = pd.read_csv(spectra_rr_csv)
display(spectra_rr_df)

print('{} linear'.format(labels_rr.count('linear')))
print('{} cyclic'.format(labels_rr.count('cyclic')))
print('{} branch-cyclic'.format(labels_rr.count('branch-cyclic')))
print('{} complex'.format(labels_rr.count('complex')))

In [ ]:
import keras

from sklearn.model_selection import train_test_split

X = spectra_df.to_numpy().astype('float32')
y = np.array(labels).astype('float32')

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.33, random_state=42)

# one-hot encode the labels
num_classes = len(np.unique(y_train))
y_train = keras.utils.to_categorical(y_train, num_classes)
y_test = keras.utils.to_categorical(y_test, num_classes)

In [ ]:
from keras.models import Sequential
from keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Dropout

model = Sequential()
model.add(Conv2D(filters=4, kernel_size=(1, 4), padding='same', activation='relu', input_shape=(1, len(discrete_masses))))
model.add(MaxPooling2D(pool_size=2))
model.add(Conv2D(filters=4, kernel_size=(1, 4), padding='same', activation='relu'))
model.add(MaxPooling2D(pool_size=2))
# model.add(Conv2D(filters=64, kernel_size=2, padding='same', activation='relu'))
# model.add(MaxPooling2D(pool_size=2))
model.add(Dropout(0.3))
model.add(Flatten())
model.add(Dense(512, activation='relu'))
model.add(Dropout(0.4))
model.add(Dense(num_classes, activation='softmax'))

model.summary()

In [ ]:
# compile the model
model.compile(loss='categorical_crossentropy', optimizer='rmsprop', metrics=['accuracy'])

In [ ]:
from keras.callbacks import ModelCheckpoint   

# train the model
checkpointer = ModelCheckpoint(filepath='model.weights.best.hdf5', verbose=1, save_best_only=True)
hist = model.fit(X_train, y_train, batch_size=32, epochs=100, 
                 validation_split=0.33, callbacks=[checkpointer], verbose=2, shuffle=True)

In [ ]:
# load the weights that yielded the best validation accuracy
model.load_weights('model.weights.best.hdf5')

In [ ]:
# evaluate and print test accuracy
score = model.evaluate(X_test, y_test, verbose=0)
print('\n', 'Test accuracy:', score[1])